# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('DatasetBasicOperations').getOrCreate()
spark

## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Remember to try letting Spark infer the header and infer the Schema types!

In [6]:
path = './data/'
df_pga = spark.read.csv(path + 'pga_tour_historical.csv', inferSchema=True, header=True)
df_pga.columns

['Player Name', 'Season', 'Statistic', 'Variable', 'Value']

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [9]:
# df_pga.limit(5).show()

df_pga.limit(5).toPandas()

,Player Name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100


## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [11]:
df_pga.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [14]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

data_types = [
    StructField(name='Player name', dataType=StringType(), nullable=True), # just showing what is each parameter
    StructField('Season', IntegerType(), True),
    StructField('Statistic', StringType(), True),
    StructField('Variable', StringType(), True),
    StructField('Value', IntegerType(), True),
]

df_pga_edited = spark.read.csv(path + 'pga_tour_historical.csv', header=True, schema=StructType(data_types))
df_pga_edited.printSchema()

root
 |-- Player name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: integer (nullable = true)



## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [19]:
df_pga_edited.select('Value').describe().show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|           1657247|
|   mean|12494.388998743096|
| stddev|157274.75673570728|
|    min|              -178|
|    max|           3564954|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [18]:
df_pga_edited.select('Season', 'Value').summary('count', 'min', 'max').show()

+-------+-------+-------+
|summary| Season|  Value|
+-------+-------+-------+
|  count|2740403|1657247|
|    min|   2010|   -178|
|    max|   2018|3564954|
+-------+-------+-------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [20]:
df_pga_filtered = df_pga_edited.select('Season', 'Value')

df_pga_filtered.write.mode('overwrite').parquet('pga_parquet')

22/02/27 20:13:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/27 20:13:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
22/02/27 20:13:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
22/02/27 20:13:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
22/02/27 20:13:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
22/02/27 20:13:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
22/02/27 20:13:20 WARN MemoryManager: Total allocation exceeds 95.

## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [21]:
df_pga_filtered.write.partitionBy('Season').parquet('pga_part_parquet')

22/02/27 20:21:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/27 20:21:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
22/02/27 20:21:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/27 20:21:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
22/02/27 20:21:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/27 20:21:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
22/02/27 20:21:33 WARN MemoryManager: Total allocation exceeds 95.00% 

## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [23]:
df_pga_part = spark.read.parquet('pga_part_parquet/')
df_pga_part.limit(5).show()

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
+-----+------+



## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [24]:
df_pga_part_2010_to_2012 = spark.read.parquet('pga_part_parquet/Season=2010', 'pga_part_parquet/Season=2011', 'pga_part_parquet/Season=2012')
df_pga_part_2010_to_2012.limit(5).show()

+-----+
|Value|
+-----+
|   71|
|   77|
|   83|
|   54|
|  100|
+-----+



In [34]:
# to show Season column you must use option basePath
basePath = 'pga_part_parquet/'
df_pga_part_2010_to_2012_opt = spark.read.option('basePath', basePath).parquet(basePath+'Season=2010/', basePath+'Season=2011/', basePath+'Season=2012/')
df_pga_part_2010_to_2012_opt.limit(5).show()

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
+-----+------+



## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [31]:
from pyspark.sql.types import BooleanType

games_personal_rate = [
    ('Devil May Cry', 'Medium', 6, False), 
    ('Sekiro - Shadows Die Twice', 'Very Hard', 9, True), 
    ('Assassins Creed Valhalla', 'Easy', 7, True)
]

columns = StructType([
    StructField('name', StringType(), True),
    StructField('difficulty', StringType()),
    StructField('score', IntegerType(), True),
    StructField('has_finished', BooleanType(), True)
])

df_games = spark.createDataFrame(games_personal_rate, columns)

df_games.toPandas()

,name,difficulty,score,has_finished
0,Devil May Cry,Medium,6,False
1,Sekiro - Shadows Die Twice,Very Hard,9,True
2,Assassins Creed Valhalla,Easy,7,True


## We're done! Great job!